In [26]:
# "If max_cols is exceeded, switch to truncate view"
pd.set_option('display.max_columns', 5400)
# "The maximum width in characters of a column"
pd.set_option('display.max_colwidth', 500)

In [25]:
import numpy as np
import codecs
import unidecode
import re
import ftfy
import pandas as pd
from wordcloud import WordCloud
import emoji
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
from spacy.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()
import spacy
import sklearn
from num2words import num2words

nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

plt.style.use('ggplot')

In [75]:
# export df that for reuse
outpath = '../data'
filename = "df_jigsaw_validation_20200307.csv"
filename_output = 'df_jigsaw_validation_20200307.csv'

In [38]:
df_jigsaw_prep = pd.read_csv(outpath + "/" + filename, usecols = ['comment_text', 'hate'],
                        encoding='utf-8')

In [ ]:
df_jigsaw_prep

In [52]:
# before extracting hashtags from tweets, we need to ride of html parts which are 
# in tweets like: &#128111;&#128131;&#128131;
import html
df_jigsaw_prep['tweet_no_html'] = df_jigsaw_prep['comment_text'].apply(html.unescape)
#df_tweets_all['tweet_lower'] = df_tweets_all['tweet_no_html'].apply(str.lower)

In [54]:
# before start text cleaning lets look at used hashtags
# function to collect hashtags
def hashtag_extract(x):
    x = x.str.lower()
    hashtags = []
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)
    return hashtags

In [55]:
def emojis_extract(x):
    emojies = []
    # Loop over the words in the tweet
    for i in x:
        em = re.findall(r":(\w+):", i)
        emojies.append(em)
    return emojies

In [56]:
def wrap_emojies(emoji_list):
    emojies = []
    for emo in emoji_list:
        emojies.append(emoji.emojize(emo))
    return emojies

In [57]:
pattern_retweet = r'^rt '
pattern_char = r'@[A-Za-z0-9_]+'
pattern_html = r'https?://[^ ]+'
pattern_combi = r'|'.join((pattern_char, pattern_html))
pattern_www = r'www.[^ ]+'
negations_dic = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "luv" : "love", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "yo" : "you",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have", "yu" : "you" }
pattern_neg = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

In [58]:
pattern_replace_url = re.compile(r'(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
old = 'hello world this https://www.gooogle.ch is a sentence www.google.com with google.ch urls http://www.bleuwin.ch'

new_ip = 'replaced_url'

replaced_url = re.sub(pattern_replace_url, new_ip, old)

print('replaced = %s' %(replaced_url))

replaced = hello world this replaced_url is a sentence replaced_url with replaced_url urls replaced_url


In [59]:
def replace_url(input_text):
    replaced_url = re.sub(pattern_replace_url, new_ip, input_text)
    return replaced_url

In [60]:
replace_url(old)

'hello world this replaced_url is a sentence replaced_url with replaced_url urls replaced_url'

In [61]:
def preprocess(input_text):
    #print("raw text: \n{} \n".format(input_text))
    decoded = ftfy.fix_encoding(input_text)
    #print("decoded: \n{} \n".format(decoded))
    #lower = decoded.lower()
    #print("lower: \n{} \n".format(lower))
    #html_free = re.sub(pattern_combi, '', lower)
    #url_free = re.sub(pattern_www, '', html_free)
    #print("html and url free: \n{}\n".format(url_free))
    neg_handled = pattern_neg.sub(lambda x: negations_dic[x.group()], decoded)
    #print("neg_handled: \n{}\n".format(neg_handled))
    demo = emoji.demojize(neg_handled)
    #to do sonderzeichen remove
    # remove special characters, numbers, punctuations aber erst nach rettung der emojies -> new function
    #combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
    ## remove twitter handles (@user)
    #combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*")
    return demo
    #stripped = re.sub(combined_pat, '', bom_removed)
    #stripped = re.sub(www_pat, '', stripped)
    #lower_case = stripped.lower()
    #neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    #letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)

In [62]:
def remove_signs(input_text):
    letters_only = re.sub("[^a-zA-Z]", " ", input_text)
    n_rt = re.sub(pattern_retweet, ' ', letters_only)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(n_rt) if len(x) > 1]
    #stemmed = [x for x  in stemmer.stem(words)]
    #stemmed = [stemmer.stem(word) for word in words]
    #print(stemmed)
    #print(words)
    return (" ".join(words)).strip()

In [63]:
def lemm(input_text):
    doc = nlp(input_text)
    # Extract the lemma for each token and join
    return(" ".join([token.lemma_ for token in doc]))

In [64]:
def spacy_cleaner(input_text):
    try:
        decoded = unidecode.unidecode(codecs.decode(input_text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(input_text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([negations_dic[t] if t in negations_dic else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [65]:
def remove_stopwords(input_text, stopword_list):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in input_text.split() if word.lower() not in stopword_list]
    # joining the list of words with space separator
    return " ".join(text)

In [66]:
df_jigsaw_prep['no_url'] = df_jigsaw_prep['tweet_no_html'].apply(replace_url)

In [67]:
df_jigsaw_prep['tweet_tidy_1'] = df_jigsaw_prep['no_url'].apply(preprocess)

In [68]:
print('done')

done


In [69]:
int(df_jigsaw_prep['tweet_tidy_1'].str.encode(encoding='utf-8').str.len().max())

5000

In [70]:
int(df_jigsaw_prep['tweet_tidy_1'].str.encode(encoding='utf-8').str.len().min())

20

In [76]:
df_jigsaw_prep.to_csv(outpath + "/" + filename_output)